In [1]:
# Autoreload imports
%load_ext autoreload
%autoreload 2

# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import plotly.express as px

from ecodynelec.parameter import Parameter  # Import the class to manipulate parameters
from ecodynelec.preprocessing.downloading import download #import the class to download entsoe data
from examples.mix_analysis.analysis_functions import format_data_0, get_metrics, plot_hourly_heatmap, plot_years, \
    plot_typical_days

# change plt font size
plt.rcParams.update({'font.size': 12})

In [2]:
# EcoDynElec configuration
my_config = Parameter()
my_config.freq = "H"
my_config.ctry = ['AT', 'CH', 'DE', 'FR', 'IT']
#my_config.target = ['CH']  # You can put multiple targets here
# Enable this if you want to plot fig 6 (will impact the computation time - it took me 15min)
my_config.target = ['AT', 'CH', 'DE', 'FR', 'IT']
my_config.data_cleaning = True
my_config.sg_imports = True

# Indicate where to save generation data
my_config.path.generation = "./test_data/downloads/generations" 
# Indicate where to save exchange data
my_config.path.exchanges = "./test_data/downloads/exchanges/"
# Recommended config: include swiss grid residual (difference between swiss grid and entso-e data)
my_config.residual_global = True
# Recommended config: include real (2020-2022) and estimated (2016-2019) renewable production (using EcoDynElec-Enr-Model)
my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2024_completed.csv'
#my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2023_completed.csv'
# Years to study
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

In [3]:
from examples.mix_analysis.analysis_functions import load_data

flows, prods, mixs, prod_impacts, impacts = load_data(my_config.target, my_config.freq, years,
                                                      savedir="./mix_analysis/results_local_all_enr/")

In [4]:
flows_CH = [flows[i]['CH'] for i in flows.keys()]
flows_CH=pd.concat(flows_CH)


flows_AT = [flows[i]['AT'] for i in flows.keys()]
flows_AT=pd.concat(flows_AT)


flows_DE = [flows[i]['DE'] for i in flows.keys()]
flows_DE=pd.concat(flows_DE)


flows_FR = [flows[i]['FR'] for i in flows.keys()]
flows_FR=pd.concat(flows_FR)


flows_IT = [flows[i]['IT'] for i in flows.keys()]
flows_IT=pd.concat(flows_IT)

In [5]:
flows_IT

,production,imports,exports
2016-01-01 00:00:00,19268.000000,3603.8920,167.910000
2016-01-01 01:00:00,18067.000000,3246.0800,150.810000
2016-01-01 02:00:00,17122.000000,3264.3928,93.320000
2016-01-01 03:00:00,16313.000000,3480.9678,114.100000
2016-01-01 04:00:00,16194.000000,3625.4790,114.750000
...,...,...,...
2024-12-31 19:00:00,24828.484894,5206.4604,14.636000
2024-12-31 20:00:00,21791.265137,5571.5590,16.220000
2024-12-31 21:00:00,19282.755524,6119.8760,0.226000
2024-12-31 22:00:00,17904.950592,6613.5010,15.459000


In [6]:
from examples.mix_analysis.analysis_functions import concatenate_and_format_data

# NOTE: Unless you use multiple targets (see section 0.), these dicts will contain only one element, corresponding to the target country
raw_productions_by_src, raw_consumptions_by_src, electricity_prod_mixs, electricity_mixs, producing_electricity_impacts, electricity_impacts = concatenate_and_format_data(my_config.target, years, flows, prods, mixs, prod_impacts, impacts)

In [7]:
# Sauvegarder les impacts d'électricité pour l'Italie (déjà existant)
electricity_impact_by_src_IT = electricity_impacts['IT']['raw_df']


# Sauvegarder les impacts d'électricité pour l'Autriche (AT)
electricity_impact_by_src_AT = electricity_impacts['AT']['raw_df']


# Sauvegarder les impacts d'électricité pour la Suisse (CH)
electricity_impact_by_src_CH = electricity_impacts['CH']['raw_df']


# Sauvegarder les impacts d'électricité pour la France (FR)
electricity_impact_by_src_FR = electricity_impacts['FR']['raw_df']


# Sauvegarder les impacts d'électricité pour l'Allemagne (DE)
electricity_impact_by_src_DE = electricity_impacts['DE']['raw_df']

In [8]:
electricity_impact_by_src_DE

,Mix_Other,Residual_Hydro_Water_Reservoir_AT,Residual_Hydro_Run-of-river_and_poundage_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,...,Other_renewable_IT,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT,Energy_storage_AT,Energy_storage_CH,Energy_storage_DE,Energy_storage_FR,Energy_storage_IT
2016-01-01 00:00:00,9.480828,0.0,0.0,0.0,0.000495,0.0,0.0,0.007220,0.002496,0.0,...,0.0,7.124071e-08,0.0,0.0,4.113415e-06,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,9.635539,0.0,0.0,0.0,0.000314,0.0,0.0,0.004537,0.001597,0.0,...,0.0,0.000000e+00,0.0,0.0,2.552429e-06,NaN,NaN,NaN,NaN,NaN
2016-01-01 02:00:00,9.825601,0.0,0.0,0.0,0.000150,0.0,0.0,0.002071,0.000773,0.0,...,0.0,0.000000e+00,0.0,0.0,5.066368e-07,NaN,NaN,NaN,NaN,NaN
2016-01-01 03:00:00,8.924160,0.0,0.0,0.0,0.000267,0.0,0.0,0.003518,0.001372,0.0,...,0.0,0.000000e+00,0.0,0.0,1.038991e-06,NaN,NaN,NaN,NaN,NaN
2016-01-01 04:00:00,8.954698,0.0,0.0,0.0,0.000382,0.0,0.0,0.004931,0.001963,0.0,...,0.0,0.000000e+00,0.0,0.0,1.657567e-06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,13.805692,0.0,0.0,0.0,0.000953,0.0,0.0,0.036987,0.000000,0.0,...,0.0,0.000000e+00,0.0,0.0,3.502740e-08,0.0,0.0,0.0,0.0,0.0
2024-12-31 20:00:00,16.223881,0.0,0.0,0.0,0.000295,0.0,0.0,0.010525,0.000000,0.0,...,0.0,0.000000e+00,0.0,0.0,1.224520e-07,0.0,0.0,0.0,0.0,0.0
2024-12-31 21:00:00,17.320877,0.0,0.0,0.0,0.000241,0.0,0.0,0.008287,0.000000,0.0,...,0.0,0.000000e+00,0.0,0.0,1.611084e-09,0.0,0.0,0.0,0.0,0.0
2024-12-31 22:00:00,18.204968,0.0,0.0,0.0,0.000422,0.0,0.0,0.014345,0.000000,0.0,...,0.0,0.000000e+00,0.0,0.0,3.246651e-07,0.0,0.0,0.0,0.0,0.0


In [9]:
tot_consumption_CH = raw_consumptions_by_src['CH']['df']


# Sauvegarder la consommation totale pour l'Autriche (AT)
tot_consumption_AT = raw_consumptions_by_src['AT']['df']


# Sauvegarder la consommation totale pour la France (FR)
tot_consumption_FR = raw_consumptions_by_src['FR']['df']

# Sauvegarder la consommation totale pour l'Allemagne (DE)
tot_consumption_DE = raw_consumptions_by_src['DE']['df']


# Sauvegarder la consommation totale pour l'Italie (IT)
tot_consumption_IT = raw_consumptions_by_src['IT']['df']

In [10]:
tot_consumption_IT

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,58.742504,1014.913475,704.638216,1327.485863,19149.887947,448.313918,22703.981922
2016-01-01 01:00:00,56.281714,925.795928,679.110998,1071.120723,17956.200514,473.759971,21162.269848
2016-01-01 02:00:00,73.008258,1010.544276,784.786151,780.807530,17053.976835,589.951202,20293.074253
2016-01-01 03:00:00,59.134831,1008.986648,685.510790,1203.681833,16229.093101,493.460503,19679.867707
2016-01-01 04:00:00,56.202080,1035.623307,661.376058,1365.601688,16110.321668,475.605517,19704.730318
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,288.905498,1996.695239,1039.092527,1481.210302,24819.970465,394.437744,30020.311775
2024-12-31 20:00:00,337.845932,1716.261360,1218.301729,1595.145289,21782.369801,696.680446,27346.604558
2024-12-31 21:00:00,342.586733,1828.153047,1309.893929,1730.230926,19282.640071,908.900864,25402.405569
2024-12-31 22:00:00,255.386785,2052.485236,1359.494321,1974.685778,17897.048723,963.891690,24502.992533


In [11]:
raw_consumption_by_src_IT = raw_consumptions_by_src['IT']['raw_df']


# Sauvegarder la consommation brute pour l'Autriche (AT)
raw_consumption_by_src_AT = raw_consumptions_by_src['AT']['raw_df']


# Sauvegarder la consommation brute pour la Suisse (CH)
raw_consumption_by_src_CH = raw_consumptions_by_src['CH']['raw_df']


# Sauvegarder la consommation brute pour la France (FR)
raw_consumption_by_src_FR = raw_consumptions_by_src['FR']['raw_df']


# Sauvegarder la consommation brute pour l'Allemagne (DE)
raw_consumption_by_src_DE = raw_consumptions_by_src['DE']['raw_df']

In [12]:
raw_consumption_by_src_DE

,Mix_Other,Residual_Hydro_Water_Reservoir_AT,Residual_Hydro_Run-of-river_and_poundage_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,...,Other_renewable_IT,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT,Energy_storage_AT,Energy_storage_CH,Energy_storage_DE,Energy_storage_FR,Energy_storage_IT
2016-01-01 00:00:00,1124.900270,0.0,0.0,0.0,0.226879,0.0,0.0,0.538080,0.112553,0.0,...,0.000000,0.000043,0.001981,0.0,0.009386,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,1124.232333,0.0,0.0,0.0,0.141511,0.0,0.0,0.332458,0.070802,0.0,...,0.000000,0.000000,0.000944,0.0,0.005728,NaN,NaN,NaN,NaN,NaN
2016-01-01 02:00:00,1131.749545,0.0,0.0,0.0,0.066872,0.0,0.0,0.149837,0.033837,0.0,...,0.000000,0.000000,0.000193,0.0,0.001122,NaN,NaN,NaN,NaN,NaN
2016-01-01 03:00:00,1044.731272,0.0,0.0,0.0,0.120637,0.0,0.0,0.258646,0.061042,0.0,...,0.000000,0.000000,0.000486,0.0,0.002339,NaN,NaN,NaN,NaN,NaN
2016-01-01 04:00:00,1028.263257,0.0,0.0,0.0,0.169538,0.0,0.0,0.355635,0.085673,0.0,...,0.000000,0.000000,0.000673,0.0,0.003661,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,2016.824609,0.0,0.0,0.0,0.537574,0.0,0.0,3.393714,0.000000,0.0,...,0.000000,0.000000,0.000109,0.0,0.000098,0.0,0.0,0.0,0.0,0.000000
2024-12-31 20:00:00,2279.294827,0.0,0.0,0.0,0.159971,0.0,0.0,0.928706,0.000000,0.0,...,0.000000,0.000000,0.000345,0.0,0.000331,0.0,0.0,0.0,0.0,0.000000
2024-12-31 21:00:00,2392.955847,0.0,0.0,0.0,0.128867,0.0,0.0,0.719098,0.000000,0.0,...,0.000000,0.000000,0.000004,0.0,0.000004,0.0,0.0,0.0,0.0,0.000000
2024-12-31 22:00:00,2483.248493,0.0,0.0,0.0,0.222494,0.0,0.0,1.228975,0.000000,0.0,...,0.000000,0.000000,0.000769,0.0,0.000852,0.0,0.0,0.0,0.0,0.000000


In [13]:
# Sauvegarder la production totale pour l'Italie (IT) - déjà existant
tot_production_IT = raw_productions_by_src['IT']['df']


# Sauvegarder la production totale pour l'Autriche (AT)
tot_production_AT = raw_productions_by_src['AT']['df']


# Sauvegarder la production totale pour la Suisse (CH)
tot_production_CH = raw_productions_by_src['CH']['df']


# Sauvegarder la production totale pour la France (FR)
tot_production_FR = raw_productions_by_src['FR']['df']


# Sauvegarder la production totale pour l'Allemagne (DE)
tot_production_DE = raw_productions_by_src['DE']['df']

In [14]:
tot_production_DE

,DE,sum
2016-01-01 00:00:00,48235.307274,48235.307274
2016-01-01 01:00:00,47771.397944,47771.397944
2016-01-01 02:00:00,47615.552137,47615.552137
2016-01-01 03:00:00,47392.680643,47392.680643
2016-01-01 04:00:00,46317.940983,46317.940983
...,...,...
2024-12-31 19:00:00,57190.071056,57190.071056
2024-12-31 20:00:00,54823.584851,54823.584851
2024-12-31 21:00:00,53564.440340,53564.440340
2024-12-31 22:00:00,53546.674238,53546.674238


In [15]:
production_IT = raw_productions_by_src['IT']['raw_df']


# Sauvegarder la production brute pour l'Autriche (AT)
production_AT = raw_productions_by_src['AT']['raw_df']


# Sauvegarder la production brute pour la Suisse (CH)
production_CH = raw_productions_by_src['CH']['raw_df']

# Sauvegarder la production brute pour la France (FR)
production_FR = raw_productions_by_src['FR']['raw_df']


# Sauvegarder la production brute pour l'Allemagne (DE)
production_DE = raw_productions_by_src['DE']['raw_df']

In [16]:
production_DE

,Residual_Hydro_Water_Reservoir_DE,Residual_Hydro_Run-of-river_and_poundage_DE,Residual_Other_DE,Biomass_DE,Fossil_Brown_coal/Lignite_DE,Fossil_Coal-derived_gas_DE,Fossil_Gas_DE,Fossil_Hard_coal_DE,Fossil_Oil_DE,Fossil_Oil_shale_DE,...,Hydro_Water_Reservoir_DE,Marine_DE,Nuclear_DE,Other_fossil_DE,Other_renewable_DE,Solar_DE,Waste_DE,Wind_Offshore_DE,Wind_Onshore_DE,Energy_storage_DE
2016-01-01 00:00:00,0.0,0.0,0.0,4782.597796,14792.639389,373.450008,1934.717608,4127.602628,199.089992,0.0,...,109.242499,0.0,10701.759746,334.030000,238.199993,0.335000,647.275011,2292.850278,6425.839825,NaN
2016-01-01 01:00:00,0.0,0.0,0.0,4767.739753,14855.259253,449.924948,1933.827363,3882.067166,199.090000,0.0,...,118.407497,0.0,10604.284424,327.367469,240.102497,0.302500,651.919960,2236.142458,6429.017710,NaN
2016-01-01 02:00:00,0.0,0.0,0.0,4776.017683,14953.163313,486.475021,1925.035237,3669.935136,199.089992,0.0,...,127.017502,0.0,10682.612780,323.887538,239.390012,0.307500,645.289996,2014.792690,6534.885128,NaN
2016-01-01 03:00:00,0.0,0.0,0.0,4771.184829,14930.383991,498.524999,1942.232687,3657.227415,199.090000,0.0,...,90.520004,0.0,10608.714488,317.594999,239.544986,0.297500,650.037461,1945.787328,6519.580024,NaN
2016-01-01 04:00:00,0.0,0.0,0.0,4773.486657,14964.463752,506.999932,1948.482469,3686.272017,199.089975,0.0,...,100.887488,0.0,10467.864381,312.917446,238.947484,0.317500,646.577484,1583.259729,5870.764796,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.0,0.0,0.0,4225.082818,5656.235144,523.957518,7500.407289,3775.140288,379.910012,0.0,...,21.344998,0.0,0.000000,168.792522,97.049993,6.855000,735.030051,1425.150114,30934.340288,0.0
2024-12-31 20:00:00,0.0,0.0,0.0,4136.942324,4381.032320,520.942510,6480.740123,3307.040034,379.909995,0.0,...,19.280001,0.0,0.000000,161.695008,97.330000,6.707499,729.530042,1206.935143,31875.759930,0.0
2024-12-31 21:00:00,0.0,0.0,0.0,4060.140156,4321.072081,515.757504,5894.042401,3111.802416,379.910003,0.0,...,25.817499,0.0,0.000000,161.759994,97.827496,7.052500,726.174978,1531.089947,30866.378436,0.0
2024-12-31 22:00:00,0.0,0.0,0.0,4038.365139,3460.460019,557.757490,4891.572757,3096.564984,379.910025,0.0,...,72.987502,0.0,0.000000,161.727497,97.902500,6.402500,717.749962,2308.484790,31953.373867,0.0


In [17]:
tot_electricity_mix_IT = electricity_mixs['IT']['df']


# Sauvegarder le mix électrique total pour l'Autriche (AT)
tot_electricity_mix_AT = electricity_mixs['AT']['df']


# Sauvegarder le mix électrique total pour la Suisse (CH)
tot_electricity_mix_CH = electricity_mixs['CH']['df']


# Sauvegarder le mix électrique total pour la France (FR)
tot_electricity_mix_FR = electricity_mixs['FR']['df']


# Sauvegarder le mix électrique total pour l'Allemagne (DE)
tot_electricity_mix_DE = electricity_mixs['DE']['df']

In [18]:
tot_electricity_mix_DE

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,0.000047,0.000817,0.962751,0.010880,1.879632e-05,0.025486,1.0
2016-01-01 01:00:00,0.000029,0.000485,0.962236,0.011339,8.929324e-06,0.025902,1.0
2016-01-01 02:00:00,0.000014,0.000193,0.966127,0.007251,1.975675e-06,0.026413,1.0
2016-01-01 03:00:00,0.000024,0.000412,0.957216,0.018354,4.133330e-06,0.023990,1.0
2016-01-01 04:00:00,0.000035,0.000639,0.954787,0.020460,6.223155e-06,0.024072,1.0
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.000167,0.000087,0.954546,0.008087,1.561190e-07,0.037112,1.0
2024-12-31 20:00:00,0.000051,0.000189,0.941141,0.015006,4.421643e-07,0.043613,1.0
2024-12-31 21:00:00,0.000042,0.000153,0.930502,0.022741,4.682559e-09,0.046561,1.0
2024-12-31 22:00:00,0.000062,0.000475,0.928224,0.022300,7.846936e-07,0.048938,1.0


In [19]:
tot_electricity_impact_IT = electricity_impacts['IT']['df']


# Sauvegarder l'impact électrique total pour l'Autriche (AT)
tot_electricity_impact_AT = electricity_impacts['AT']['df']


# Sauvegarder l'impact électrique total pour la Suisse (CH)
tot_electricity_impact_CH = electricity_impacts['CH']['df']


# Sauvegarder l'impact électrique total pour la France (FR)
tot_electricity_impact_FR = electricity_impacts['FR']['df']


# Sauvegarder l'impact électrique total pour l'Allemagne (DE)
tot_electricity_impact_DE = electricity_impacts['DE']['df']

In [20]:
tot_electricity_impact_DE

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,0.011051,0.010599,522.005743,0.428125,0.022491,9.480828,531.958837
2016-01-01 01:00:00,0.006976,0.005364,521.180094,0.454852,0.010709,9.635539,531.293534
2016-01-01 02:00:00,0.003247,0.002112,523.091545,0.298325,0.002361,9.825601,533.223191
2016-01-01 03:00:00,0.005601,0.004362,519.842101,0.775890,0.004955,8.924160,529.557069
2016-01-01 04:00:00,0.007914,0.006771,531.807142,0.844825,0.007415,8.954698,541.628765
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.044571,0.001516,301.140233,0.227339,0.000090,13.805692,315.219441
2024-12-31 20:00:00,0.014221,0.002405,261.525583,0.397019,0.000256,16.223881,278.163366
2024-12-31 21:00:00,0.011925,0.002019,258.718146,0.627823,0.000003,17.320877,276.680793
2024-12-31 22:00:00,0.015538,0.005229,230.432841,0.642467,0.000458,18.204968,249.301501


In [21]:
src_mapping = {'Mix_Other' : 'Other',
'Residual_Hydro_Run-of-river_and_poundage': 'Hydro run-of-river',
'Biomass' :'Biomass' ,
'Fossil_Coal-derived_gas': 'Gas',
'Fossil_Oil':'Oil',
'Geothermal' :'Geothermal',
'Hydro_Run-of-river_and_poundage' :'Hydro run-of-river',
'Marine': 'Other',
'Solar' : 'Solar',
'Residual_Hydro_Water_Reservoir' : 'Hydro lake',
'Residual_Other': 'Other',
'Fossil_Brown_coal/Lignite':'Coal',
'Fossil_Gas': 'Gas',
'Fossil_Oil_shale':'Oil',
'Hydro_Pumped_Storage' : 'Hydro storage',
'Hydro_Water_Reservoir' : 'Hydro lake',
'Nuclear' : 'Nuclear',
'Waste' : 'Waste',
'Fossil_Hard_coal':'Coal',
'Fossil_Peat':'Coal',
'Other_fossil': 'Other',
'Wind_Offshore' : 'Wind',
'Other_renewable': 'Other',
'Wind_Onshore' : 'Wind'}


def group_by_src(DF):
    # Supposons que df soit votre DataFrame
    df = DF.copy()
    df.columns = [col[:-3] if col != 'Mix_Other' else col for col in df.columns]
    df = df.groupby(df.columns, axis=1).sum()
    df = pd.concat([df, df[['Mix_Other']]], axis=1)
    df = df.rename(columns=src_mapping)
    df = df.groupby(df.columns, axis=1).sum()

    return df

In [22]:
Techno_impact_IT = group_by_src(electricity_impact_by_src_IT)


Techno_impact_AT = group_by_src(electricity_impact_by_src_AT)


Techno_impact_CH = group_by_src(electricity_impact_by_src_CH)

Techno_impact_FR = group_by_src(electricity_impact_by_src_FR)


Techno_impact_DE = group_by_src(electricity_impact_by_src_DE)

# Pour l'Italie (IT)
Techno_IT = group_by_src(raw_consumption_by_src_IT)


Techno_AT = group_by_src(raw_consumption_by_src_AT)


Techno_CH = group_by_src(raw_consumption_by_src_CH)


Techno_FR = group_by_src(raw_consumption_by_src_FR)


Techno_DE = group_by_src(raw_consumption_by_src_DE)

C:\Users\PC Jules\AppData\Local\Temp\ipykernel_20300\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_20300\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_20300\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_20300\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_20300\653172070.py:31: FutureWarning: DataFrame.groupby with axis

In [23]:
Techno_CH

,Biomass,Coal,Energy_storage,Gas,Geothermal,Hydro lake,Hydro run-of-river,Hydro storage,Nuclear,Oil,Other,Solar,Waste,Wind
2016-01-01 00:00:00,256.838423,939.250276,0.000000,209.310508,3.664409,541.649366,525.893725,199.649301,3297.541912,14.763639,657.712811,0.058388,39.532902,456.191759
2016-01-01 01:00:00,270.069572,980.331599,0.000000,218.312381,3.261536,542.778949,541.179787,100.270650,3397.963207,15.176421,682.637394,0.047154,41.997500,471.426824
2016-01-01 02:00:00,277.408995,988.738608,0.000000,219.072561,2.223873,515.679778,538.093037,92.779163,3302.879465,14.413491,725.803326,0.039659,43.328665,466.228803
2016-01-01 03:00:00,246.810083,882.106499,0.000000,198.436809,2.192487,512.679179,519.816470,83.519211,3359.510525,13.554528,620.208054,0.036142,38.693439,411.393486
2016-01-01 04:00:00,241.004928,865.325723,0.000000,193.397908,2.170862,533.940163,527.043569,83.896039,3379.158920,13.395358,588.253220,0.051938,37.484214,355.880928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,202.876562,343.017323,0.000000,434.499388,1.340648,964.029377,994.179275,672.083065,2423.594429,15.025956,501.861722,0.653821,36.281064,1311.914671
2024-12-31 20:00:00,236.967572,336.579613,0.000000,473.994967,1.633803,615.066970,980.233267,362.132513,2628.139661,18.163047,672.673515,0.604527,44.190796,1615.598711
2024-12-31 21:00:00,239.963727,338.671958,0.000000,449.167359,1.456824,739.848561,870.267492,408.270013,2604.107055,18.844951,704.572872,0.548064,45.192288,1648.088617
2024-12-31 22:00:00,278.060019,351.909909,0.000000,485.254185,1.977260,1074.021796,1480.614448,326.886940,2881.858342,22.394277,889.028128,0.507237,53.331237,2061.582147


## Sauvegarde des DataFrames au format parquet.gz

In [24]:
base_dir = r'C:\Users\PC Jules\Desktop\UI\Ecodynelec\data'

#

folders = {
    'consumptions': {
        'raw_consumption_by_src_AT': raw_consumption_by_src_AT,
        'raw_consumption_by_src_CH': raw_consumption_by_src_CH,
        'raw_consumption_by_src_DE': raw_consumption_by_src_DE,
        'raw_consumption_by_src_FR': raw_consumption_by_src_FR,
        'raw_consumption_by_src_IT': raw_consumption_by_src_IT,
        'tot_consumption_AT': tot_consumption_AT,
        'tot_consumption_CH': tot_consumption_CH,
        'tot_consumption_DE': tot_consumption_DE,
        'tot_consumption_FR': tot_consumption_FR,
        'tot_consumption_IT': tot_consumption_IT
    },
    'electricity_impacts': {
        'electricity_impact_AT': tot_electricity_impact_AT,
        'electricity_impact_CH': tot_electricity_impact_CH,
        'electricity_impact_DE': tot_electricity_impact_DE,
        'electricity_impact_FR': tot_electricity_impact_FR,
        'electricity_impact_IT': tot_electricity_impact_IT,
        'electricity_impact_by_src_AT': electricity_impact_by_src_AT,
        'electricity_impact_by_src_CH': electricity_impact_by_src_CH,
        'electricity_impact_by_src_DE': electricity_impact_by_src_DE,
        'electricity_impact_by_src_FR': electricity_impact_by_src_FR,
        'electricity_impact_by_src_IT': electricity_impact_by_src_IT
    },
    'electricity_mixs': {
        'electricity_mix_AT': tot_electricity_mix_AT,
        'electricity_mix_CH': tot_electricity_mix_CH,
        'electricity_mix_DE': tot_electricity_mix_DE,
        'electricity_mix_FR': tot_electricity_mix_FR,
        'electricity_mix_IT': tot_electricity_mix_IT
    },
    'flows': {
        'flows_AT': flows_AT,
        'flows_CH': flows_CH,
        'flows_DE': flows_DE,
        'flows_FR': flows_FR,
        'flows_IT': flows_IT
    },
    'productions': {
        'production_AT': production_AT,
        'production_CH': production_CH,
        'production_DE': production_DE,
        'production_FR': production_FR,
        'production_IT': production_IT,
        'tot_production_AT': tot_production_AT,
        'tot_production_CH': tot_production_CH,
        'tot_production_DE': tot_production_DE,
        'tot_production_FR': tot_production_FR,
        'tot_production_IT': tot_production_IT
    },
    'technologies': {
        'Techno_impact_AT': Techno_impact_AT,
        'Techno_impact_CH': Techno_impact_CH,
        'Techno_impact_DE': Techno_impact_DE,
        'Techno_impact_FR': Techno_impact_FR,
        'Techno_impact_IT': Techno_impact_IT,
        'technologies_AT': Techno_AT,
        'technologies_CH': Techno_CH,
        'technologies_DE': Techno_DE,
        'technologies_FR': Techno_FR,
        'technologies_IT': Techno_IT
    }
}

for folder_name, dataframes in folders.items():
    folder_path = os.path.join(base_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    for file_name, df in dataframes.items():
        file_path = os.path.join(folder_path, f'{file_name}.parquet.gz')
        df.to_parquet(file_path, compression='gzip')
        print(f'Saved: {file_path}')

print('\nToutes les données ont été sauvegardées avec succès!')

Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\raw_consumption_by_src_AT.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\raw_consumption_by_src_CH.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\raw_consumption_by_src_DE.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\raw_consumption_by_src_FR.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\raw_consumption_by_src_IT.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\tot_consumption_AT.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\tot_consumption_CH.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\tot_consumption_DE.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\tot_consumption_FR.parquet.gz
Saved: C:\Users\PC Jules\Desktop\UI\Ecodynelec\data\consumptions\tot_consumption_IT.parquet.gz
Saved: C:\Users

KeyboardInterrupt: 